# Tiny Airlift — Base vs. BQP (QUBO) Formulations
This notebook presents two stylized formulations for a toy Airlift planning task:

1) **Base (Enumeration)**: Full decision enumeration on a very small instance.
2) **BQP/QUBO**: A binary quadratic program that selects one route plan and a set of delivered cargos; solved here by brute force.

Both versions use the **same tiny instance** and objective priorities: minimize missed deliveries, then lateness, then flight cost.

Additional information can be found on the origional [project site](https://airliftchallenge.com/chapters/main.html).

The goal is start by understanding this simplified model and then move on to the full challenge. 

## 0) Tiny Instance
- Airports: `A, B, C`
- Routes (time, cost): A→B (1,5), B→C (1,5), C→B (1,5), A→C (2,9), C→A (2,9), B→A (1,5)
- Plane: start `A`, capacity 10
- Cargos:
  - c1: A→C, weight 6, soft=2, hard=3
  - c2: B→A, weight 4, soft=2, hard=3
- Horizon H=4 (discrete steps t=0..3).

In [ ]:

# Instance
AIRPORTS = ["A", "B", "C"]
ROUTES = {
    ("A","B"): {"time":1, "cost":5},
    ("B","C"): {"time":1, "cost":5},
    ("C","B"): {"time":1, "cost":5},
    ("A","C"): {"time":2, "cost":9},
    ("C","A"): {"time":2, "cost":9},
    ("B","A"): {"time":1, "cost":5},
}
PLANE_CAPACITY = 10
PLANE_START = "A"
CARGOS = {
    "c1": {"origin":"A", "dest":"C", "weight":6, "soft":2, "hard":3},
    "c2": {"origin":"B", "dest":"A", "weight":4, "soft":2, "hard":3},
}
H = 4
# Scoring
M1 = 10_000  # missed-delivery penalty
M2 = 100     # per-step lateness beyond soft deadline
print("OK: instance loaded")

## 1) Base Version — Enumerate Decisions and Simulate
At each time step, if the plane is at an airport it may unload, load (subject to capacity), and optionally depart. If enroute, it continues flying. We enumerate all feasible choices and pick the plan with minimal score.

In [ ]:

from collections import namedtuple

Action = namedtuple("Action", ["move_to", "load", "unload"])  # move_to None=stay or airport code; load/unload tuples of cargo ids

def outgoing_from(a):
    return [b for (x,b) in ROUTES if x==a]

def enumerate_actions(current_airport, onboard, at_airport_cargos):
    unloadable = [c for c in onboard if CARGOS[c]["dest"]==current_airport]
    # Enumerate unload subsets
    for u_mask in range(1<<len(unloadable)):
        unload = [unloadable[i] for i in range(len(unloadable)) if (u_mask>>i)&1]
        # Loadable = waiting here whose origin is here (no transfers in this tiny base model)
        loadable = [c for c in at_airport_cargos if CARGOS[c]["origin"]==current_airport]
        for l_mask in range(1<<len(loadable)):
            load = [loadable[i] for i in range(len(loadable)) if (l_mask>>i)&1]
            # Capacity check after unload then load
            carry = set(onboard) - set(unload)
            wt = sum(CARGOS[c]["weight"] for c in carry) + sum(CARGOS[c]["weight"] for c in load)
            if wt > PLANE_CAPACITY:
                continue
            for m in [None] + outgoing_from(current_airport):
                yield Action(m, tuple(load), tuple(unload))

In [ ]:

def simulate(decisions):
    # cargo locations: 'airport:X', 'plane', or 'delivered'
    cargo_loc = {c: f"airport:{CARGOS[c]['origin']}" for c in CARGOS}
    delivered_time = {c: None for c in CARGOS}
    t = 0
    plane_airport = PLANE_START
    plane_onboard = set()
    enroute = None
    flight_cost = 0
    log = []

    while t < H:
        rec = {"t": t, "where": None, "action": None, "onboard": sorted(plane_onboard)}
        if enroute:
            enroute["rem"] -= 1
            rec["where"] = f"enroute→{enroute['dest']} ({enroute['rem']} left)"
            rec["action"] = "--"
            if enroute["rem"] == 0:
                plane_airport = enroute["dest"]
                enroute = None
                rec["where"] = f"arrive {plane_airport}"
        else:
            rec["where"] = f"at {plane_airport}"
            act = decisions[t]
            # unload
            for c in act.unload:
                if c in plane_onboard and CARGOS[c]["dest"]==plane_airport:
                    plane_onboard.remove(c)
                    if delivered_time[c] is None:
                        delivered_time[c] = t
            # load (only from origin and if cargo still waiting there)
            for c in act.load:
                if cargo_loc[c] == f"airport:{plane_airport}":
                    plane_onboard.add(c)
                    cargo_loc[c] = "plane"
            if act.move_to is not None:
                a, b = plane_airport, act.move_to
                if (a,b) in ROUTES:
                    leg = ROUTES[(a,b)]
                    enroute = {"rem": leg["time"], "dest": b}
                    flight_cost += leg["cost"]
                    plane_airport = None
                    rec["action"] = f"depart {a}→{b}"
                else:
                    rec["action"] = "invalid"
            else:
                rec["action"] = "stay"
        rec["onboard_after"] = sorted(plane_onboard)
        log.append(rec)
        t += 1

    # score
    missed = 0
    late = 0
    for c, info in CARGOS.items():
        s, h = info["soft"], info["hard"]
        dt = delivered_time[c]
        if dt is None or dt > h:
            missed += 1
        else:
            late += max(0, dt - s)
    score = M1*missed + M2*late + flight_cost
    # on-time set
    ontime = {c for c, dt in delivered_time.items() if (dt is not None and dt <= CARGOS[c]["hard"])}
    return score, {"log":log, "delivered_time":delivered_time, "missed":missed, "late":late, "cost":flight_cost, "ontime": ontime}

In [ ]:

# Depth-first enumeration with simulator to pick the optimal plan
def best_plan():
    def rec(t, loc, onboard, enroute, prefix):
        if t == H:
            sc, det = simulate(prefix + [Action(None,(),())]*(H-len(prefix)))
            return [(sc, prefix, det)]
        results = []
        if enroute:
            new = dict(enroute)
            new["rem"] -= 1
            new_loc = loc
            if new["rem"] == 0:
                new_loc = new["dest"]
                new = None
            results += rec(t+1, new_loc, set(onboard), new, prefix+[Action(None,(),())])
        else:
            wait_here = [c for c in CARGOS if f"airport:{loc}" == f"airport:{CARGOS[c]['origin']}"]
            for act in enumerate_actions(loc, set(onboard), wait_here):
                # progress enroute meta-state for branching
                next_enroute = None
                next_loc = loc
                if act.move_to is not None and (loc, act.move_to) in ROUTES:
                    leg = ROUTES[(loc, act.move_to)]
                    next_enroute = {"rem": leg["time"], "dest": act.move_to}
                    next_loc = None
                results += rec(t+1, next_loc if next_enroute is None else None, set(onboard), next_enroute, prefix+[act])
        return results

    all_res = rec(0, PLANE_START, set(), None, [])
    return min(all_res, key=lambda x: x[0])

best_score, best_decisions, best_details = best_plan()
best_score, best_details["missed"], best_details["late"], best_details["cost"], best_details["ontime"]

## 2) BQP/QUBO Version — Choose a Route Plan and Deliveries
We pre-generate a **small set of route plans** (from the enumeration) and capture, for each plan, which cargos are delivered **on time** and the **flight cost**. Then we build a binary quadratic model:

- Variables: `r_k` selects plan `k` (one-hot), `y_c` indicates delivering cargo `c`.
- Objective: **penalize** not-one-hot plans and any `(y_c=1)` when the selected plan can’t deliver `c` on time; reward on-time deliveries; add the plan’s flight cost.

We solve the tiny QUBO by brute force and decode the chosen plan.

In [ ]:

# Build a compact candidate set from enumeration results:
# Keep the cheapest plan for every distinct set of on-time deliveries.
from collections import defaultdict

def all_plans():
    # reuse the DFS but collect all leaf plans with their simulation details
    def rec(t, loc, onboard, enroute, prefix, acc):
        if t == H:
            sc, det = simulate(prefix + [Action(None,(),())]*(H-len(prefix)))
            acc.append((sc, prefix, det))
            return
        if enroute:
            new = dict(enroute)
            new["rem"] -= 1
            new_loc = loc
            if new["rem"] == 0:
                new_loc = new["dest"]
                new = None
            rec(t+1, new_loc, set(onboard), new, prefix+[Action(None,(),())], acc)
        else:
            wait_here = [c for c in CARGOS if f"airport:{loc}" == f"airport:{CARGOS[c]['origin']}"]
            for act in enumerate_actions(loc, set(onboard), wait_here):
                next_enroute = None
                next_loc = loc
                if act.move_to is not None and (loc, act.move_to) in ROUTES:
                    leg = ROUTES[(loc, act.move_to)]
                    next_enroute = {"rem": leg["time"], "dest": act.move_to}
                    next_loc = None
                rec(t+1, next_loc if next_enroute is None else None, set(onboard), next_enroute, prefix+[act], acc)

    acc = []
    rec(0, PLANE_START, set(), None, [], acc)
    return acc

raw = all_plans()

# For BQP, cluster by delivered_on_time set and keep cheapest by flight cost to keep K small
best_by_set = {}
for sc, decs, det in raw:
    key = frozenset(det["ontime"])
    cost = det["cost"]
    if (key not in best_by_set) or (cost < best_by_set[key]["cost"]):
        best_by_set[key] = {"score": sc, "cost": cost, "decs": decs, "details": det}

candidates = list(best_by_set.items())  # [(frozenset(...), info), ...]
# Index them
PLAN_LIST = []
for idx, (ontime_set, info) in enumerate(candidates):
    PLAN_LIST.append({
        "id": idx,
        "ontime": set(ontime_set),
        "cost": info["cost"],
        "decs": info["decs"],
        "details": info["details"],
    })

print("Candidate plans (deduped by on-time set):")
for p in PLAN_LIST:
    print(f"  Plan {p['id']}: on-time {sorted(p['ontime'])}, cost {p['cost']}")

In [ ]:

# Build and solve a QUBO:
# Variables: r_k for k in plans, y_c for cargos
# Objective (to MINIMIZE):
#   Q = P1*(sum_k r_k - 1)^2 + sum_k cost_k r_k  + P2*sum_{c,k} (1 - feas_{c,k}) y_c r_k  - R * sum_c y_c
# Here feas_{c,k}=1 iff plan k delivers c on-time.
import itertools, math

CARGOS_LIST = sorted(list(CARGOS.keys()))
K = len(PLAN_LIST)
C = len(CARGOS_LIST)

# weights
R = 500       # reward per on-time delivery (dominates cost)
P1 = 2000     # one-hot enforcement
P2 = 2000     # infeasibility enforcement

def qubo_value(r_bits, y_bits):
    # r_bits, y_bits: tuples of 0/1
    # one-hot penalty
    s = sum(r_bits)
    Q = P1 * (s - 1)**2
    # plan cost
    for k, rk in enumerate(r_bits):
        if rk:
            Q += PLAN_LIST[k]["cost"]
    # infeasibility penalty and reward
    # penalize choosing y_c=1 if chosen plan doesn't deliver c on-time
    for ci, c in enumerate(CARGOS_LIST):
        if y_bits[ci]==1:
            # reward
            Q -= R
            # infeasibility check for each selected plan r_k
            for k, rk in enumerate(r_bits):
                if rk:
                    feas = 1 if (c in PLAN_LIST[k]["ontime"]) else 0
                    if feas == 0:
                        Q += P2  # violation
    return Q

# brute-force solve
best_Q = math.inf
best_r = None
best_y = None
for r_bits in itertools.product([0,1], repeat=K):
    for y_bits in itertools.product([0,1], repeat=C):
        val = qubo_value(r_bits, y_bits)
        if val < best_Q:
            best_Q, best_r, best_y = val, r_bits, y_bits

print("Best QUBO value:", best_Q)
print("Chosen plan r:", best_r, " chosen y:", best_y)
# Decode
chosen_k = [i for i,b in enumerate(best_r) if b==1]
chosen_k = chosen_k[0] if len(chosen_k)==1 else None
chosen_plan = PLAN_LIST[chosen_k] if chosen_k is not None else None
chosen_deliveries = [CARGOS_LIST[i] for i,b in enumerate(best_y) if b==1]
chosen_k, chosen_plan["ontime"] if chosen_plan else None, chosen_deliveries

## 3) Compare BQP vs. Base Optimum
We display the base-optimal plan and the BQP-selected plan to verify they align (on this toy).

In [ ]:

import pandas as pd
from caas_jupyter_tools import display_dataframe_to_user
import matplotlib.pyplot as plt

# Base timeline
base_df = pd.DataFrame(best_details["log"])
display_dataframe_to_user("Base-Optimal Timeline", base_df)

print("Base: on-time:", best_details["ontime"], " cost:", best_details["cost"], " missed:", best_details["missed"], " late:", best_details["late"])

# BQP timeline
if chosen_plan:
    bqp_df = pd.DataFrame(chosen_plan["details"]["log"])
    display_dataframe_to_user("BQP-Selected Timeline", bqp_df)
    print("BQP: on-time:", chosen_plan["ontime"], " cost:", chosen_plan["cost"])
else:
    print("BQP did not select a unique plan (check penalties).")

# Simple alignment check
aligned = (chosen_plan is not None) and (chosen_plan["ontime"]==best_details["ontime"])
print("Alignment (on-time sets equal)?", aligned)

## 4) Notes & Extensions
- The **Base** model is an exact enumeration for this tiny instance.
- The **BQP** uses a *route-selection* abstraction: pick one plan and a set of deliveries with quadratic penalties to enforce consistency.
- To scale up:
  1) Expand candidate plans via beam search or k-shortest-time-feasible routes per aircraft.
  2) Solve the resulting QUBO with annealers or BQP solvers.
  3) Add soft-deadline penalties to the reward term (e.g., reduce `R` if only late delivery).